In [1]:
import math
import pandas as pd
import numpy as np

In [2]:


def extract_parameters_from_excel(file_path):
    # Load sheets with specified names, skipping the ID and name columns/rows for matrices
    distance_df = pd.read_excel(file_path, sheet_name='Distance (km)').iloc[1:, 2:]
    travel_time_df = pd.read_excel(file_path, sheet_name='Travel time (min)').iloc[1:, 2:]
    flow_df = pd.read_excel(file_path, sheet_name='Flow').iloc[1:, 2:]
    fixed_link_cost_df = pd.read_excel(file_path, sheet_name='Fixed link cost').iloc[1:, 2:]
    fixed_hub_cost_df = pd.read_excel(file_path, sheet_name='Fixed hub cost').iloc[:, 2:]
    
    # Convert distance and travel time dataframes to matrices
    distance = distance_df.values
    travel_time = travel_time_df.values

    # Flow matrix: create nested dictionary to handle directional flow
    flow_matrix = flow_df.values   # Use iloc to get the first column's value for the flow

    # Fixed link cost: convert to matrix
    fixed_link_cost = fixed_link_cost_df.values

    # Fixed hub cost: extract city names and costs as a dictionary
    fixed_hub_cost = fixed_hub_cost_df.values


    return distance, travel_time, flow_matrix, fixed_link_cost, fixed_hub_cost


In [3]:
file_path = 'Turkish_network.xls'
distance, travel_time, flow_matrix, fixed_linked_cost, fixed_hub_cost = extract_parameters_from_excel(file_path)


In [4]:

def find_hub(I, f, u, d, flows, hubcosts):
    min_cost = math.inf
    hub = None

    f = np.array(f)
    u = np.array(u)
    d = np.array(d)
    flows = np.array(flows)
    hubcosts = np.array(hubcosts)
    costs = []
    for i in I:
        inflow_cost = 0
        outflow_cost = 0
        # Fixed hub cost
        fixed_hub_cost = hubcosts[i]

        # Calculate inflow_cost
        for j in I:
            #print(i,j)
            if j != i and flows[j][i] > 0:
                
                inflow_cost += f[j][i] + u[j][i] * d[j][i] * flows[j][i]

        # Calculate outflow_cost
        for j in I:
            if j != i and flows[i][j] > 0:
                outflow_cost += f[i][j] + u[i][j] * d[i][j] * flows[i][j]

        
        # Total cost for hub i
        c_i = inflow_cost + outflow_cost + fixed_hub_cost
        costs.append(c_i)
        if c_i <= min_cost:
            min_cost = c_i
            hub = i

    return hub+1, min_cost, costs


In [5]:
#u = np.full((81, 81), 1e-4)
u = fixed_linked_cost * 1e-3
I = np.arange(0,81)
fixed_linkage_costs =np.zeros((81,81))
hub, min_cost, costs = find_hub(I, fixed_linkage_costs, u, distance, flow_matrix, fixed_hub_cost)

In [6]:
hub, min_cost

(38, array([1255409.07619987]))